In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [4]:
# Loading training data

df = pd.read_csv('iith_foml_2023_train.csv')


X = df.drop(columns=['Target Variable (Discrete)'])
X.fillna(X.mean(), inplace=True)
y = df['Target Variable (Discrete)']


# tried something here, but failed
# print(np.unique(y))
# expected_classes = np.arange(np.max(y) + 1)
# missing_classes = np.setdiff1d(expected_classes, np.unique(y))
# print("Missing classes:", missing_classes)

In [5]:
# Split the data into training and testing sets
# for analysing how good the model is


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [6]:
#Gradient Boost classifier
gboost_classifier = GradientBoostingClassifier(n_estimators=500, random_state=42)


# Define logistic regression for comparing
logestic_reg_classifier = LogisticRegression()


pipeline_gboost = make_pipeline(StandardScaler(), gboost_classifier)

pipeline_logestic_reg = make_pipeline(StandardScaler(), logestic_reg_classifier)

# Cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Evaluating the models 
accuracy_gb = cross_val_score(pipeline_gboost, X_train, y_train, cv=cv, scoring='accuracy')

accuracy_logreg = cross_val_score(pipeline_logestic_reg, X_train, y_train, cv=cv, scoring='accuracy')

# comparing the cross validation results
print(f"Gradient Boosting Accuracy: {np.mean(accuracy_gb)}")

print(f"Logistic Regression Accuracy: {np.mean(accuracy_logreg)}")

# Choose the best model and fit it to the entire training set
best_model = pipeline_gboost  
# choosing gradient boosting as it is better for multi classfication than logestic regression
 
best_model.fit(X_train, y_train)

# Evaluate on the test set that we made from our test data

y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
test_f1_score = f1_score(y_test, y_pred, average='weighted')

print(f"Test Accuracy: {test_accuracy}")
print(f"Test F1 Score: {test_f1_score}")

e:\Kali\Python\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
e:\Kali\Python\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
e:\Kali\Python\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
e:\Kali\Python\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1

Gradient Boosting Accuracy: 0.8993670886075951
Logistic Regression Accuracy: 0.8264240506329115
Test Accuracy: 0.8492462311557789
Test F1 Score: 0.8307270714558872


In [7]:
test_data = pd.read_csv('test_input.csv')
X_eval = test_data.iloc[:, :]
X_eval.fillna(X_eval.mean(), inplace=True)

X_eval = scaler.transform(X_eval)

In [8]:
y_eval = best_model.predict(X_eval)

In [9]:
sno = []
for i in range(len(y_eval)):
    sno.append(i+1)

In [10]:
df = pd.DataFrame({'Id': sno, 'Category': y_eval})
filename = 'test_output.csv'
df.to_csv(filename, index=False)